In [15]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt
import time 
import copy

"""
This is an EFX allocation finder using simulated annealing. 
In short, it does simulated annealing with the number of EFX violations
as an objective function. See the accompanying paper for the description of the algorithm.
"""

# --- Core helper functions ---

def _count_violations_for_pair(envious_idx, envied_idx, v, bundles, agent_utilities):
    # Counts EFX violations for a single ordered pair (i,j) using the utilities that have been precomputed.
    # Get the envious agent's utility for their own bundle from the cache (O(1) lookup)
    u_envious_own = agent_utilities[envious_idx]

    # Compute the utility for the other bundle on the fly
    envied_bundle = bundles[envied_idx]
    u_envious_of_envied = sum(v[envious_idx][item] for item in envied_bundle)

    violations = 0
    for item_k in envied_bundle:
        if u_envious_of_envied - v[envious_idx][item_k] > u_envious_own:
            violations += 1
    return violations

def calculate_full_potential(n, v, bundles, agent_utilities):
    # Calculates the total potential from scratch. Used only for initialization.
    total_violations = 0
    for i in range(n):
        for j in range(n):
            if i == j: continue
            total_violations += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
    return total_violations

# --- Initialization functions ---

def initialize_allocation(n, m, v):

    # Creates a random allocation and all necessary cached data structures.
    # Returns:
    #    bundles (list of sets): bundles[i] is the set of items for agent i.
    #    owner (list): owner[j] is the agent who owns item j.
    #    agent_utilities (list): agent_utilities[i] = u_i(A_i).
    
    bundles = [set() for _ in range(n)]
    owner = [-1] * m
    agent_utilities = [0.0] * n

    for item_j in range(m):
        agent_i = random.randint(0, n - 1)
        bundles[agent_i].add(item_j)
        owner[item_j] = agent_i
        agent_utilities[agent_i] += v[agent_i][item_j]
        
    return bundles, owner, agent_utilities

# --- Potentially better initialization: start with the welfare maximizing allocation --- 
def initialize_allocation_from_optimal_welfare(n, m, v):
    # 
    # Creates an allocation that maximizes social welfare and all necessary cached data structures.
    # Returns:
    #    bundles (list of sets): bundles[i] is the set of items for agent i.
    #    owner (list): owner[j] is the agent who owns item j.
    #    agent_utilities (list): agent_utilities[i] = u_i(A_i).
    
    bundles = [set() for _ in range(n)]
    owner = [-1] * m
    agent_utilities = [0.0] * n

    for item_j in range(m):
        agent_i = -1 # will give item_j to the agent that likes it the most
        highest_value_for_j = -1
        for i in range(n):
            if v[i][item_j] > highest_value_for_j:
                agent_i = i
                highest_value_for_j = v[i][item_j]
        
        bundles[agent_i].add(item_j)
        owner[item_j] = agent_i
        agent_utilities[agent_i] += v[agent_i][item_j]
        
    return bundles, owner, agent_utilities

def generate_random_valuations_int(n, m):
    # Generates the valuation matrix using random values.
    return [[random.uniform(1, 1000) for _ in range(m)] for _ in range(n)]

def generate_random_valuations(n, m):
    # Generates the valuation matrix using random values.
    return [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)]

def sample_correlated_valuations(n, m, correlation_strength=0.8, seed=None):
    # Samples additive valuations for agents that are correlated.
    if seed is not None:
        np.random.seed(seed)
    assert 0 <= correlation_strength <= 1, "correlation_strength must be between 0 and 1"
    shared = np.random.uniform(0, 1, size=m)
    v = [[0 for _ in range(m)] for _ in range(n)]
    for i in range(n):
        noise = np.random.uniform(0, 1, size=m)
        for j in range(m):
            v[i][j] = correlation_strength * shared[j] + (1 - correlation_strength) * noise[j]
    return v

# --- Functions to save the last instance (valuation and initial/final allocation) to a file in case assert breaks ---
# Save the valuation matrix to a file 
def write_valuation_to_file(n, m, matrix):
    with open('valuation_matrix.txt', 'w') as f:
        for row in matrix:
            # Convert each element to string and join with a separator
            row_str = ' '.join(map(str, row))
            f.write(row_str + '\n')
    return True

# Save the allocation matrix to a file 
def write_allocation_to_file(n, m, bundles, filename):
    # First we convert the allocation to matrix format
    matrix = [[0 for e in range(m)] for e in range(n)]
    for i in range(n):
        for j in bundles[i]:
            matrix[i][j] = 1
    
    with open(filename, 'w') as f:
        for row in matrix:
            # Convert each element to string and join with a separator
            row_str = ' '.join(map(str, row))
            f.write(row_str + '\n')
    return True

# --- Main simulation ---

def find_efx_allocation(n, m, v, bundles, owner, agent_utilities):
    # 1. Setup 
    
    num_steps = 0 # Count the total number of steps until reaching an EFX allocation

    num_restarts = 0
    
    while True: # Loop for restarts
        num_restarts += 1
        
        # 2. Init 
                
        f = calculate_full_potential(n, v, bundles, agent_utilities)
        
        best_f_value = f
        T = 5.0  # Initial temperature
        T_min = 0.0001
        cooling_rate = 0.99

        #print(f"\n--- Restart #{num_restarts} ---")
        #print(f"Initial potential: {f}")

        # 3. Simulated annealing loop 
        while T > T_min and best_f_value > 0:
            steps_per_temp = 100 * n * m # 10 * int(m * n / T) # number of steps at the current temperature T 
            
            for _ in range(steps_per_temp):
                if best_f_value == 0: break

                # a) Propose a random move
                item_to_move = random.randint(0, m - 1)
                owner_old = owner[item_to_move]
                owner_new = random.randint(0, n - 1)
                while owner_new == owner_old:
                    owner_new = random.randint(0, n - 1)

                # b) Calculate the change in potential (the delta)
                affected_agents = {owner_old, owner_new}
                
                # Calculate the part of the potential related to these agents before the move
                old_slice_f = 0
                for i in range(n):
                    for j in affected_agents:
                        if i != j: old_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
                for i in affected_agents:
                    for j in range(n):
                        if i != j and j not in affected_agents:
                            old_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)

                # c) Tentatively apply the move to the data structures
                bundles[owner_old].remove(item_to_move)
                bundles[owner_new].add(item_to_move)
                agent_utilities[owner_old] -= v[owner_old][item_to_move]
                agent_utilities[owner_new] += v[owner_new][item_to_move]

                # d) Calculate the part of the potential after the move
                new_slice_f = 0
                for i in range(n):
                    for j in affected_agents:
                        if i != j: new_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
                for i in affected_agents:
                    for j in range(n):
                        if i != j and j not in affected_agents:
                            new_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
                
                # e) Decide whether to accept the move
                delta_f = new_slice_f - old_slice_f
                
                accept_move = False
                if delta_f < 0:
                    accept_move = True
                else:
                    if T > 0:
                        acceptance_probability = math.exp(-delta_f / T)
                        if random.uniform(0, 1) < acceptance_probability:
                            accept_move = True
                
                # f) Finalize or revert the move
                if accept_move:
                    owner[item_to_move] = owner_new
                    f += delta_f
                    # print("Moving to an allocation with function value", f)

                else:
                    # Revert all changes
                    bundles[owner_new].remove(item_to_move)
                    bundles[owner_old].add(item_to_move)
                    agent_utilities[owner_new] -= v[owner_new][item_to_move]
                    agent_utilities[owner_old] += v[owner_old][item_to_move]
                
                if f < best_f_value:
                    best_f_value = f
                
                num_steps += 1

            if best_f_value == 0: break # Found EFX allocation 
            T *= cooling_rate
        
        print(f"End of SA run. Best potential found: {best_f_value}. Total steps: {num_steps}")
        if best_f_value == 0:
            print("\n>>> EFX allocation found! <<<")
            break

    # Save the final allocation to file for cross-check
    #write_allocation_to_file(n, m, bundles, 'allocation_final_matrix.txt')

    potential_recalculated = calculate_full_potential(n, v, bundles, agent_utilities)
    assert potential_recalculated == 0, "Recalculated the potential at the end and it is not zero"

    return num_steps, bundles, owner, agent_utilities # for accounting purposes return how many steps it took, the allocation and the auxiliary data structures

    #print(f"\nTotal restarts required: {num_restarts}")

# This implements the algorithm that lets the agents repeatedly choose items in their favorite order;
# i.e. agent 1 chooses their favorite item, then agent 2 chooses their favorite item among the remaining ones, then agent 3, ..., agent n, 
# agent 1, ... 
# This does not always lead to EFX allocation on random instances and the algorithm fails very often with correlated valuations
def round_robin_allocation(n, m, v):

  # Let the agents choose their favorite items in round robin fashion
  bundles = [[] for _ in range(n)]
  owner = [-1] * m
  agent_utilities = [0.0] * n

  items_remaining = []
  for j in range(m):
    items_remaining.append(j)

  while len(items_remaining) > 0:
    # let current_agent take his favorite item from the remaining ones
    for i in range(n): # each agent takes their remaining item

      if len(items_remaining) == 0: break # no more items to be allocated

      # sort the items in decreasing order of value according to the preferences of agent i
      items_remaining.sort(key=lambda p: v[i][int(p)], reverse = True)
      j = items_remaining[0]
      bundles[i].append(j)
      owner[j] = i
      agent_utilities[i] += v[i][j]

      items_remaining.remove(j)

  return bundles, owner, agent_utilities

# check if an allocation is exactly envy-free
def is_envy_free(n, v, bundles, agent_utilities):
    # Checks if the allocation is actually envy-free

    for i in range(n):
        for j in range(n):
            if i == j: continue

            u_i = utilities[i] # utility of agent i for its own bundle

            u_ij = 0 # utility of agent i for agent j's bundle

            for k in bundles[j]:
              u_ij += v[i][k]

            if u_ij > u_i: # Agent i envies agent j
              return False

    return True # the allocation is envy-free

if __name__ == '__main__':

    # --- Initialization  ---

    n_values = [4]
    m_values = [25]

    with open("test", 'w') as f:

        for n in n_values:
            for m in m_values:        
                    print("n = " + str(n) + "\n") # Use f.write to save to the file instead
                    print("m = " + str(m) + "\n")
                    #f.flush()                

                    T = 1
                    run_time = [0 for e in range(T)]
                    num_steps = [0 for e in range(T)]

                    for R in range(T):
                        v = generate_random_valuations(n, m)    
                        print("Generated valuation:", v)
                        
                        bundles, owner, agent_utilities = initialize_allocation(n, m, v)

                        print("Initialized the allocation to::", bundles)
                        start_time = time.time()
                        num_steps[R], bundles_end, owner_end, agent_utilities_end = find_efx_allocation(n, m, v, bundles, owner, agent_utilities) 
                        end_time = time.time()
                        total_time = end_time - start_time     
                        print("The final allocation is::", bundles_end)
                
                        print("Finished experiment #" + str(R) + " in " + str(total_time) + " seconds\n")
                        print("Finished experiment #" + str(R) + " in " + str(num_steps[R]) + " steps\n")
                        #f.flush()
                
                        run_time[R] = end_time - start_time

                    avg_runtime = sum(run_time) / T
                    std_dev = np.std(run_time)

                    avg_num_steps = sum(num_steps) / T
                    std_dev_steps = np.std(num_steps)
            
                    #latex_output = (
                    #    f"{{{n}}}   &  "
                    #    f"\\makecell{{{avg_runtime:.2f} $\\pm$ {std_dev:.2f}}} & "
                    #    f"\\makecell{{{avg_num_steps:.0f} $\\pm$ {std_dev_steps:.0f}}} \\\\ \\hline"
                    #)
                    #f.write(latex_output)
                    #f.write("\n")
                    #f.flush()
            

n = 5

m = 35

Generated valuation: [[0.8014747509080163, 0.09834178273514138, 0.5665349955660302, 0.30245680831523203, 0.14640303045697922, 0.07897826033688271, 0.11754043140148995, 0.6487323967452221, 0.7247468077829295, 0.9366127318641978, 0.30328732404599446, 0.012740205808109972, 0.5283289443516643, 0.5384080689973939, 0.0584455723196593, 0.24549148884590533, 0.09077742852720094, 0.1717129226826244, 0.6701073169854753, 0.3638723129473179, 0.9441623156176835, 0.10840540935651433, 0.1790875753968012, 0.5302005104030874, 0.42423099327503544, 0.20633365752793775, 0.15071318378636167, 0.4938028358740396, 0.8257795353338261, 0.46661428441707753, 0.44696574344900697, 0.8566398959539085, 0.8167951274459204, 0.5395044328214017, 0.29821599041382785], [0.24057908783359438, 0.6688670351576748, 0.39017044575141113, 0.6744843566451327, 0.483671130875019, 0.9347537017032143, 0.12092374315458687, 0.8506909979807337, 0.9302106354594586, 0.49011009445611564, 0.2485029247526549, 0.23817839874337354,